Este código implementa uma decision tree, usando diferentes critérios para escolher os melhores atributos e diferentes métodos para resolver conflitos, fazer Pre-Prunning e Pos-Prunning.

Os critérios de seleção de atributos disponíveis são  Entropy, Gini Index e Gain Ratio, enquanto os métodos de resolução de conflitos são Poda, Majority voting e Class threshold.

A pre-Prunning pode ser feita com base em Size, Maximum Depth ou Independence, enquanto a pos-Prunning pode ser feita com base em  Pessimistic error prunning ou Reduced error prunning.

O código implementa a construção da árvore de decisão recursivamente e, em cada nó, o melhor atributo é escolhido para dividir os dados, de acordo com o critério selecionado. A impureza antes e depois da divisão é calculada e o ganho de informação é obtido a partir desses valores. Se o ganho de informação não atingir um determinado threshold, a folha é criada para esse nó. Se um dos subconjuntos resultantes da divisão for vazio, a folha é criada para esse nó.

In [1]:
import numpy as np
from typing import Optional

def _gini( y_pred):
    _, counts = np.unique(y_pred, return_counts=True)
    probs = counts / len(y_pred)
    return 1 - np.sum(probs ** 2)


def _entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probs = counts / len(y)
    return -np.sum(probs * np.log2(probs))


def _split(feature, y, threshold):
    left_indices = np.where(feature <= threshold)[0]
    right_indices = np.where(feature > threshold)[0]
    return left_indices, right_indices


def gain_ratio(X, y, feature_indices):
    base_impurity = _entropy(y)
    max_gain_ratio = 0
    best_feature_index = None
    best_threshold = None
    for feature_index in feature_indices:
        values = np.unique(X[:, feature_index])
        for threshold in values:
            left_indices, right_indices = _split(X[:, feature_index], y, threshold)
            left_ratio = len(left_indices) / len(X)
            right_ratio = len(right_indices) / len(X)
            split_impurity = left_ratio * _entropy(y[left_indices]) + right_ratio * _entropy(y[right_indices])
            information_gain = base_impurity - split_impurity
            split_info = -left_ratio * np.log2(left_ratio) - right_ratio * np.log2(right_ratio)
            gain_r = information_gain / split_info if split_info != 0 else 0
            if gain_r > max_gain_ratio:
                max_gain_ratio = gain_r
                best_feature_index = feature_index
                best_threshold = threshold
    return (best_feature_index, best_threshold) if max_gain_ratio > 0 else None


def select_best_split_entropy(X, y, feature_indices):
    best_feature_index, best_threshold, best_info_gain = None, None, -float("inf")

    for feature_index in feature_indices:
        thresholds = np.unique(X[:, feature_index])
        for threshold in thresholds:
            left_indices = X[:, feature_index] <= threshold
            right_indices = X[:, feature_index] > threshold
            H_y_x = (np.sum(left_indices) / len(y)) * _entropy(y[left_indices]) + \
                    (np.sum(right_indices) / len(y)) * _entropy(y[right_indices])
            info_gain = _entropy(y) - H_y_x
            if info_gain > best_info_gain:
                best_feature_index = feature_index
                best_threshold = threshold
                best_info_gain = info_gain

    return best_feature_index, best_threshold


def select_best_split_gini_index(X, y, feature_indices):
    best_feature_index = None
    best_threshold = None
    best_gini_index = float('inf')
    for feature_index in feature_indices:
        unique_thresholds = np.unique(X[:, feature_index])
        for threshold in unique_thresholds:
            left_indices = X[:, feature_index] <= threshold
            right_indices = X[:, feature_index] > threshold
            if len(left_indices) == 0 or len(right_indices) == 0:
                continue
            left_gini = _gini(y[left_indices])
            right_gini = _gini(y[right_indices])
            current_gini_index = (left_gini * np.sum(left_indices) + right_gini * np.sum(right_indices)) / len(y)
            if current_gini_index < best_gini_index:
                best_gini_index = current_gini_index
                best_feature_index = feature_index
                best_threshold = threshold
    return best_feature_index, best_threshold


def split_information(n_left, n_right):
    n_total = n_left + n_right
    p_left = n_left / n_total
    p_right = n_right / n_total
    if p_left == 0 or p_right == 0:
        return 0
    else:
        return - p_left * np.log2(p_left) - p_right * np.log2(p_right)


def calculate_gain_ratio(y, left_indices, right_indices):
    entropy_before_split = _entropy(y)

    left_entropy = _entropy(y[left_indices])
    right_entropy = _entropy(y[right_indices])
    n_instances = len(y)
    n_left = len(left_indices)
    n_right = n_instances - n_left
    weighted_avg_entropy = (n_left / n_instances) * left_entropy + (n_right / n_instances) * right_entropy

    split_info = split_information(n_left, n_right)

    if split_info == 0:
        gain_ratio = 0
    else:
        gain_ratio = (entropy_before_split - weighted_avg_entropy) / split_info

    return gain_ratio


def select_best_split_gain_ratio(X, y, feature_indices):
    best_feature_index, best_threshold, max_gain_ratio = None, None, 0.0
    H_y = _entropy(y)
    for feature_index in feature_indices:
        thresholds = np.unique(X[:, feature_index])
        for threshold in thresholds:
            left_indices = np.where(X[:, feature_index] <= threshold)[0].astype(int)
            right_indices = np.where(X[:, feature_index] > threshold)[0].astype(int)
            if len(left_indices) == 0 or len(right_indices) == 0:
                continue
            H_y_x = (len(left_indices) / len(y)) * _entropy(y[left_indices]) + \
                    (len(right_indices) / len(y)) * _entropy(y[right_indices])
            split_info = split_information(len(left_indices), len(right_indices))
            gain_ratio = (H_y - H_y_x) / split_info if split_info != 0 else 0
            if gain_ratio > max_gain_ratio:
                best_feature_index, best_threshold, max_gain_ratio = feature_index, threshold, gain_ratio
    return best_feature_index, best_threshold


def building_tree(X, y, feature_indices, max_depth=None, min_samples_leaf=1, impurity_measure="entropy"):
    if len(np.unique(y)) == 1:
        return Node(label=y[0])
    if max_depth is not None and max_depth == 0:
        return Node(label=np.argmax(np.bincount(y)))
    if len(X) < min_samples_leaf:
        return Node(label=np.argmax(np.bincount(y)))
    if impurity_measure == "entropy":
        impurity_func = _entropy
        best_criteria_func = select_best_split_entropy
    elif impurity_measure == "gini_index":
        impurity_func = _gini
        best_criteria_func = select_best_split_gini_index
    elif impurity_measure == "gain_ratio":
        impurity_func = _entropy
        best_criteria_func = select_best_split_gain_ratio
    else:
        raise ValueError("Invalid impurity measure specified")
    best_feature_index, best_threshold = best_criteria_func(X, y, feature_indices)
    if best_feature_index is None:
        return Node(label=np.argmax(np.bincount(y)))
    left_indices = np.where(X[:, best_feature_index] <= best_threshold)[0].astype(int)
    right_indices = np.where(X[:, best_feature_index] > best_threshold)[0].astype(int)
    left_child = building_tree(X[left_indices], y[left_indices], feature_indices, max_depth=max_depth-1 if max_depth is not None else None, min_samples_leaf=min_samples_leaf, impurity_measure=impurity_measure)
    right_child = building_tree(X[right_indices], y[right_indices], feature_indices, max_depth=max_depth-1 if max_depth is not None else None, min_samples_leaf=min_samples_leaf, impurity_measure=impurity_measure)
    return Node(split_feature=best_feature_index, split_threshold=best_threshold, left_child=left_child,
                right_child=right_child, impurity=impurity_func(y))


def prunning(node, X, y, prune_method, threshold=None):
    if node.label is not None:
        return node
    
    node.left_child = prunning(node.left_child, X, y, prune_method, threshold)
    node.right_child = prunning(node.right_child, X, y, prune_method, threshold)
    
    left_label = None
    right_label = None
    
    if node.left_child.label is not None and node.right_child.label is not None:
        if prune_method == "majority_voting":
            left_label = node.left_child.label
            right_label = node.right_child.label
            
            if np.sum(y == left_label) > np.sum(y == right_label):
                node.label = left_label
            else:
                node.label = right_label
        elif prune_method == "class_threshold":
            node_samples = len(y)
            
            if node.left_child.label is not None:
                left_label = node.left_child.label
                left_samples = len(np.where(y[node.left_child.indices] == left_label)[0])
            else:
                left_samples = 0
                
            if node.right_child.label is not None:
                right_label = node.right_child.label
                right_samples = len(np.where(y[node.right_child.indices] == right_label)[0])
            else:
                right_samples = 0
                
            if (left_samples / node_samples >= threshold) and (right_samples / node_samples >= threshold):
                node.label = None
                
    return node


def predict(X, node):
    y_pred = []
    for x in X:
        current_node = node
        while current_node.label is None:
            if x[current_node.split_feature] <= current_node.split_threshold:
                current_node = current_node.left_child
            else:
                current_node = current_node.right_child
        y_pred.append(current_node.label)
    return np.array(y_pred)

class DecisionTree:
    def __init__(self, 
                 max_depth: Optional[int] = None,
                 min_samples_leaf: int = 1,
                 impurity_measure: str = "entropy",
                 attribute_selection: str = "gain_ratio",
                 prune_method: Optional[str] = None,
                 threshold: Optional[float] = None) -> None:
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.impurity_measure = impurity_measure
        self.attribute_selection = attribute_selection
        self.prune_method = prune_method
        self.threshold = threshold

    def fit(self, X: np.ndarray, y: np.ndarray) -> "DecisionTree":
        self.n_features_ = X.shape[1]
        self.feature_indices_ = np.arange(self.n_features_)
        if self.attribute_selection == "random":
            self.feature_indices_ = np.random.choice(self.feature_indices_, size=int(np.sqrt(self.n_features_)),
                                                     replace=False)
        self.root_ = building_tree(X, y, self.feature_indices_, self.max_depth, self.min_samples_leaf,
                                self.impurity_measure)
        if self.prune_method is not None:
            self.root_ = prunning(self.root_, X, y, self.prune_method, self.threshold)
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        return predict(X, self.root_)

class Node:
    def __init__(self, **kwargs):
        self.split_feature = kwargs.get('split_feature', None)
        self.split_threshold = kwargs.get('split_threshold', None)
        self.left_child = kwargs.get('left_child', None)
        self.right_child = kwargs.get('right_child', None)
        self.label = kwargs.get('label', None)
        self.impurity = kwargs.get('impurity', None)


**Exemplos de Aplicação**

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

value = DecisionTree(max_depth=10, min_samples_leaf=20, impurity_measure="entropy",
                             attribute_selection="gain_ratio", prune_method="reduced_error", threshold=0.2)

value.fit(X_train, y_train)

y_pred = value.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.93


In [3]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_digits(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

value = DecisionTree(max_depth=10, min_samples_leaf=20, impurity_measure="entropy",
                             attribute_selection="gain_ratio", prune_method="reduced_error", threshold=0.2)

value.fit(X_train, y_train)

y_pred = value.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.86
